In [ ]:
#Validación cruzada

import os
import torch
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision import transforms
from sklearn.model_selection import LeaveOneGroupOut
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Dispositivo:", device)


class MyoSegDataset2D(Dataset):
    def __init__(self, image_root, mask_root):
        self.image_paths, self.mask_paths, self.patients = [], [], []
        for patient in sorted(os.listdir(image_root)):
            img_folder = os.path.join(image_root, patient)
            mask_folder = os.path.join(mask_root, patient)
            if not os.path.isdir(mask_folder):
                print(f"⚠️ Falta máscara para {patient}")
                continue

            imgs = sorted([f for f in os.listdir(img_folder) if f.endswith('.png')])
            masks = sorted([f for f in os.listdir(mask_folder) if f.endswith('.png')])
            for i, m in zip(imgs, masks):
                self.image_paths.append(os.path.join(img_folder, i))
                self.mask_paths.append(os.path.join(mask_folder, m))
                self.patients.append(patient)

        self.transform = transforms.ToTensor()

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img = Image.open(self.image_paths[idx]).convert("L")
        mask = Image.open(self.mask_paths[idx]).convert("L")
        img = self.transform(img)
        mask = (self.transform(mask) > 0).float()
        return img, mask, self.image_paths[idx], self.patients[idx]

import torch.nn as nn

class DoubleConv2D(nn.Module):
    def __init__(self, in_ch, out_ch, p=0.1):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
            nn.Dropout2d(p)
        )

    def forward(self, x):
        return self.conv(x)

class UNet2D(nn.Module):
    def __init__(self, in_channels=1, out_channels=1, base_filters=64, p=0.1):
        super().__init__()
        self.enc1 = DoubleConv2D(in_channels, base_filters, p)
        self.enc2 = DoubleConv2D(base_filters, base_filters*2, p)
        self.enc3 = DoubleConv2D(base_filters*2, base_filters*4, p)
        self.pool = nn.MaxPool2d(2)
        self.bottom = DoubleConv2D(base_filters*4, base_filters*8, p)
        self.up2 = nn.ConvTranspose2d(base_filters*8, base_filters*4, 2, stride=2)
        self.dec2 = DoubleConv2D(base_filters*8, base_filters*4, p)
        self.up1 = nn.ConvTranspose2d(base_filters*4, base_filters*2, 2, stride=2)
        self.dec1 = DoubleConv2D(base_filters*4, base_filters*2, p)
        self.up0 = nn.ConvTranspose2d(base_filters*2, base_filters, 2, stride=2)
        self.dec0 = DoubleConv2D(base_filters*2, base_filters, p)
        self.out_conv = nn.Conv2d(base_filters, out_channels, 1)

    def forward(self, x):
        e1 = self.enc1(x)
        e2 = self.enc2(self.pool(e1))
        e3 = self.enc3(self.pool(e2))
        b = self.bottom(self.pool(e3))
        d2 = self.dec2(torch.cat([self.up2(b), e3], dim=1))
        d1 = self.dec1(torch.cat([self.up1(d2), e2], dim=1))
        d0 = self.dec0(torch.cat([self.up0(d1), e1], dim=1))
        return torch.sigmoid(self.out_conv(d0))


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# === Rutas necesarias ===
image_root = "/kaggle/input/dataset/New-UNet2D/img_2D/img_train"
mask_root = "/kaggle/input/dataset/New-UNet2D/mask_2D/mask_train"

print("📁 Contenido de las rutas:")
print("image_root:", image_root)
print("mask_root:", mask_root)
print("Pacientes con imagen:", os.listdir(image_root))
print("Pacientes con máscara:", os.listdir(mask_root))

# === Cargar dataset
dataset = MyoSegDataset2D(image_root, mask_root)

# === Imprimir información del dataset
print(f"📊 Total muestras cargadas: {len(dataset)}")
print(f"👥 Pacientes detectados: {set(dataset.patients)}")

# === Cargar dataset y modelo ===
dataset = MyoSegDataset2D(image_root, mask_root)
patient_ids = np.array(dataset.patients)
logo = LeaveOneGroupOut()

# Identificar índices de train/val para fold 5
FOLD_ID = 4  # Fold 5 → índice 4 (Python index starts at 0)
splits = list(logo.split(np.arange(len(dataset)), groups=patient_ids))
train_idx, val_idx = splits[FOLD_ID]

# Crear loader de entrenamiento
train_subset = Subset(dataset, train_idx)
train_loader = DataLoader(train_subset, batch_size=1, shuffle=False)

# Cargar modelo
model = UNet2D().to(device)
model.load_state_dict(torch.load("/kaggle/input/best-model/unet2d_best_fold5.pth"))
model.eval()

# Crear carpeta para guardar predicciones si quieres
os.makedirs("predicciones_train_fold5", exist_ok=True)

# === Inferencia sobre pacientes de entrenamiento ===
with torch.no_grad():
    for i, (img, mask, path, patient_id) in tqdm(enumerate(train_loader), total=len(train_loader)):
        img, mask = img.to(device), mask.to(device)
        pred = model(img)
        pred_bin = (pred > 0.5).float()

        # Visualización rápida
        pred_np = pred.squeeze().cpu().numpy()
        overlay = np.clip(img.squeeze().cpu().numpy() + pred_np, 0, 1)

        fig, axs = plt.subplots(1, 3, figsize=(12, 4))
        axs[0].imshow(img.squeeze().cpu(), cmap="gray"); axs[0].set_title("Imagen")
        axs[1].imshow(mask.squeeze().cpu(), cmap="gray"); axs[1].set_title("Máscara GT")
        axs[2].imshow(overlay, cmap="hot"); axs[2].set_title("Predicción")
        for ax in axs: ax.axis("off")
        plt.tight_layout()
        plt.savefig(f"predicciones_train_fold5/pred_{i:03d}.png")
        plt.close()

from torch.utils.data import Subset


Dispositivo: cuda
📁 Contenido de las rutas:
image_root: /kaggle/input/dataset/New-UNet2D/img_2D/img_train
mask_root: /kaggle/input/dataset/New-UNet2D/mask_2D/mask_train
Pacientes con imagen: ['2851790', '1297164', '1405347', '1637400', '6554609', '1741355']
Pacientes con máscara: ['2851790', '1297164', '1405347', '1637400', '6554609', '1741355']
📊 Total muestras cargadas: 8064
👥 Pacientes detectados: {'1297164', '2851790', '6554609', '1637400', '1405347', '1741355'}


 45%|████▍     | 3022/6720 [19:25<22:13,  2.77it/s]  

In [ ]:
#Predicciones del paciente de validación del fold 5

import os
import torch
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision import transforms
from sklearn.model_selection import LeaveOneGroupOut
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Dispositivo:", device)



class MyoSegDataset2D(Dataset):
    def __init__(self, image_root, mask_root):
        self.image_paths, self.mask_paths, self.patients = [], [], []
        for patient in sorted(os.listdir(image_root)):
            img_folder = os.path.join(image_root, patient)
            mask_folder = os.path.join(mask_root, patient)
            if not os.path.isdir(mask_folder): continue
            imgs = sorted([f for f in os.listdir(img_folder) if f.endswith('.png')])
            masks = sorted([f for f in os.listdir(mask_folder) if f.endswith('.png')])
            for i, m in zip(imgs, masks):
                self.image_paths.append(os.path.join(img_folder, i))
                self.mask_paths.append(os.path.join(mask_folder, m))
                self.patients.append(patient)

        self.transform = transforms.ToTensor()

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img = Image.open(self.image_paths[idx]).convert("L")
        mask = Image.open(self.mask_paths[idx]).convert("L")
        img = self.transform(img)
        mask = (self.transform(mask) > 0).float()
        return img, mask, self.image_paths[idx], self.patients[idx]

import torch.nn as nn

class DoubleConv2D(nn.Module):
    def __init__(self, in_ch, out_ch, p=0.1):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
            nn.Dropout2d(p)
        )

    def forward(self, x):
        return self.conv(x)

class UNet2D(nn.Module):
    def __init__(self, in_channels=1, out_channels=1, base_filters=64, p=0.1):
        super().__init__()
        self.enc1 = DoubleConv2D(in_channels, base_filters, p)
        self.enc2 = DoubleConv2D(base_filters, base_filters*2, p)
        self.enc3 = DoubleConv2D(base_filters*2, base_filters*4, p)
        self.pool = nn.MaxPool2d(2)
        self.bottom = DoubleConv2D(base_filters*4, base_filters*8, p)
        self.up2 = nn.ConvTranspose2d(base_filters*8, base_filters*4, 2, stride=2)
        self.dec2 = DoubleConv2D(base_filters*8, base_filters*4, p)
        self.up1 = nn.ConvTranspose2d(base_filters*4, base_filters*2, 2, stride=2)
        self.dec1 = DoubleConv2D(base_filters*4, base_filters*2, p)
        self.up0 = nn.ConvTranspose2d(base_filters*2, base_filters, 2, stride=2)
        self.dec0 = DoubleConv2D(base_filters*2, base_filters, p)
        self.out_conv = nn.Conv2d(base_filters, out_channels, 1)

    def forward(self, x):
        e1 = self.enc1(x)
        e2 = self.enc2(self.pool(e1))
        e3 = self.enc3(self.pool(e2))
        b = self.bottom(self.pool(e3))
        d2 = self.dec2(torch.cat([self.up2(b), e3], dim=1))
        d1 = self.dec1(torch.cat([self.up1(d2), e2], dim=1))
        d0 = self.dec0(torch.cat([self.up0(d1), e1], dim=1))
        return torch.sigmoid(self.out_conv(d0))


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



# === Rutas necesarias ===
image_root = "/kaggle/input/dataset-2d/New-UNet2D/img_2D/img_train"
mask_root = "/kaggle/input/dataset-2d/New-UNet2D/mask_2D/mask_train"

# === Cargar dataset y modelo ===
dataset = MyoSegDataset2D(image_root, mask_root)
patient_ids = np.array(dataset.patients)
logo = LeaveOneGroupOut()

# Identificar índices de train/val para fold 5
FOLD_ID = 4  # Fold 5 → índice 4 (Python index starts at 0)
splits = list(logo.split(np.arange(len(dataset)), groups=patient_ids))
train_idx, val_idx = splits[FOLD_ID]

# Crear loader de entrenamiento
train_subset = Subset(dataset, train_idx)
train_loader = DataLoader(train_subset, batch_size=1, shuffle=False)

# Cargar modelo
model = UNet2D().to(device)
model.load_state_dict(torch.load("/kaggle/input/checkpoint/unet2d_best_fold5.pth"))
model.eval()

# Crear carpeta para guardar predicciones si quieres
os.makedirs("predicciones_train_fold5", exist_ok=True)

# === Inferencia sobre pacientes de entrenamiento ===
with torch.no_grad():
    for i, (img, mask, path, patient_id) in tqdm(enumerate(train_loader), total=len(train_loader)):
        img, mask = img.to(device), mask.to(device)
        pred = model(img)
        pred_bin = (pred > 0.5).float()

        # Visualización rápida
        pred_np = pred.squeeze().cpu().numpy()
        overlay = np.clip(img.squeeze().cpu().numpy() + pred_np, 0, 1)

        fig, axs = plt.subplots(1, 3, figsize=(12, 4))
        axs[0].imshow(img.squeeze().cpu(), cmap="gray"); axs[0].set_title("Imagen")
        axs[1].imshow(mask.squeeze().cpu(), cmap="gray"); axs[1].set_title("Máscara GT")
        axs[2].imshow(overlay, cmap="hot"); axs[2].set_title("Predicción")
        for ax in axs: ax.axis("off")
        plt.tight_layout()
        plt.savefig(f"predicciones_train_fold5/pred_{i:03d}.png")
        plt.close()

from torch.utils.data import Subset

In [ ]:
#Predicción del paciente de validación para cada fold en cada caso

import os
import torch
import numpy as np
from PIL import Image
from tqdm import tqdm
from torch.utils.data import Subset, DataLoader
from torchvision import transforms
from sklearn.model_selection import LeaveOneGroupOut

# === Dataset personalizado
class MyoSegDataset2D(torch.utils.data.Dataset):
    def __init__(self, image_root, mask_root):
        self.image_paths, self.mask_paths, self.patients = [], [], []
        for patient in sorted(os.listdir(image_root)):
            img_folder = os.path.join(image_root, patient)
            mask_folder = os.path.join(mask_root, patient)
            if not os.path.isdir(mask_folder):
                continue
            imgs = sorted([f for f in os.listdir(img_folder) if f.endswith('.png')])
            masks = sorted([f for f in os.listdir(mask_folder) if f.endswith('.png')])
            for i, m in zip(imgs, masks):
                self.image_paths.append(os.path.join(img_folder, i))
                self.mask_paths.append(os.path.join(mask_folder, m))
                self.patients.append(patient)
        self.transform = transforms.ToTensor()

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img = Image.open(self.image_paths[idx]).convert("L")
        mask = Image.open(self.mask_paths[idx]).convert("L")
        img = self.transform(img)
        mask = (self.transform(mask) > 0).float()
        return img, mask, self.image_paths[idx], self.patients[idx]

# === UNet2D con Dropout y BatchNorm
class DoubleConv2D(torch.nn.Module):
    def __init__(self, in_ch, out_ch, p=0.1):
        super().__init__()
        self.conv = torch.nn.Sequential(
            torch.nn.Conv2d(in_ch, out_ch, 3, padding=1),
            torch.nn.BatchNorm2d(out_ch),
            torch.nn.ReLU(inplace=True),
            torch.nn.Conv2d(out_ch, out_ch, 3, padding=1),
            torch.nn.BatchNorm2d(out_ch),
            torch.nn.ReLU(inplace=True),
            torch.nn.Dropout2d(p)
        )

    def forward(self, x):
        return self.conv(x)

class UNet2D(torch.nn.Module):
    def __init__(self, in_channels=1, out_channels=1, base_filters=64, p=0.1):
        super().__init__()
        self.enc1 = DoubleConv2D(in_channels, base_filters, p)
        self.enc2 = DoubleConv2D(base_filters, base_filters*2, p)
        self.enc3 = DoubleConv2D(base_filters*2, base_filters*4, p)
        self.pool = torch.nn.MaxPool2d(2)
        self.bottom = DoubleConv2D(base_filters*4, base_filters*8, p)
        self.up2 = torch.nn.ConvTranspose2d(base_filters*8, base_filters*4, 2, stride=2)
        self.dec2 = DoubleConv2D(base_filters*8, base_filters*4, p)
        self.up1 = torch.nn.ConvTranspose2d(base_filters*4, base_filters*2, 2, stride=2)
        self.dec1 = DoubleConv2D(base_filters*4, base_filters*2, p)
        self.up0 = torch.nn.ConvTranspose2d(base_filters*2, base_filters, 2, stride=2)
        self.dec0 = DoubleConv2D(base_filters*2, base_filters, p)
        self.out_conv = torch.nn.Conv2d(base_filters, out_channels, 1)

    def forward(self, x):
        e1 = self.enc1(x)
        e2 = self.enc2(self.pool(e1))
        e3 = self.enc3(self.pool(e2))
        b = self.bottom(self.pool(e3))
        d2 = self.dec2(torch.cat([self.up2(b), e3], dim=1))
        d1 = self.dec1(torch.cat([self.up1(d2), e2], dim=1))
        d0 = self.dec0(torch.cat([self.up0(d1), e1], dim=1))
        return torch.sigmoid(self.out_conv(d0))

# === CONFIGURACIÓN
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
image_root = "/kaggle/input/dataset-2d/New-UNet2D/img_2D/img_train"
mask_root = "/kaggle/input/dataset-2d/New-UNet2D/mask_2D/mask_train"
model_dir = "/kaggle/input/predicciones"  # donde están los .pth del fold1 al 4

# === Dataset y partición Leave-One-Group-Out
dataset = MyoSegDataset2D(image_root, mask_root)
patient_ids = np.array(dataset.patients)
logo = LeaveOneGroupOut()
splits = list(logo.split(np.arange(len(dataset)), groups=patient_ids))

# === Inference para folds 1 a 6
for fold_id in range(6):  # folds 0 a 5 → Fold1 a Fold6
    print(f"\n🔄 Iniciando predicción para Fold {fold_id + 1}")
    
    _, val_idx = splits[fold_id]
    val_loader = DataLoader(Subset(dataset, val_idx), batch_size=1, shuffle=False)

    # Cargar modelo correspondiente
    model = UNet2D().to(device)
    model_path = os.path.join(model_dir, f"unet2d_best_fold{fold_id + 1}.pth")
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()

    # Crear carpeta para guardar predicciones
    pred_dir = f"predicciones_fold{fold_id + 1}_val"
    os.makedirs(pred_dir, exist_ok=True)

    with torch.no_grad():
        for img, _, path, patient_id in tqdm(val_loader, desc=f"Inferencia Fold {fold_id + 1}"):
            img = img.to(device)
            pred = model(img)
            pred_bin = (pred > 0.5).float().squeeze().cpu().numpy() * 255

            fname = os.path.basename(path[0])
            patient = patient_id[0]
            out_folder = os.path.join(pred_dir, patient)
            os.makedirs(out_folder, exist_ok=True)
            Image.fromarray(pred_bin.astype(np.uint8)).save(os.path.join(out_folder, fname))

    print(f"✅ ¡Predicciones del fold {fold_id + 1} guardadas en: {pred_dir}")

In [ ]:
import shutil

# Crear ZIP para cada carpeta de predicciones de fold
for fold_id in range(1, 7):  # folds 1 al 6
    folder = f"predicciones_fold{fold_id}_val"
    zip_name = f"{folder}.zip"
    shutil.make_archive(folder, 'zip', folder)
    print(f"✅ Carpeta '{folder}' comprimida como '{zip_name}'")